In [ ]:
import glob
import os
import numpy as np
import nltk
from nltk.stem import PorterStemmer

In [ ]:
#taking input from stop word list file
fObj=open('StopwordList.txt','r')
SwContent=fObj.readlines()
swlist = [x.replace("\n","").replace(" ","") for x in SwContent]
swl = [x for x in swlist if x!=""] # stop word list stored in 'swl'

In [ ]:
# print(swl)
ldict={}

In [ ]:
porter = PorterStemmer()

In [ ]:
list=glob.glob('Speeches/*')
# print(list)
for x in list:
    f=open(x,'r')
    fullfile=f.read().replace(".","").replace("n't"," not").replace("'","").replace("]"," ").replace("[","").replace(","," ").replace("?","").replace("\n"," ").replace("-"," ").split() 
    lowCasedoc=[porter.stem(x.lower()) for x in fullfile]
    withoutstoplist = [x for x in lowCasedoc if x not in swl]
#     print(len(withoutstoplist))
    p=os.path.basename(x)
    p=p.split('.')[0]
    ldict[p]=withoutstoplist

In [ ]:
print(ldict.keys())
# temp = 'Speeches\\speech_0.txt'
# p = os.path.basename(temp)
# print(p.split('.')[0])

In [ ]:
index_table={}
for key in ldict.keys():
    for word in set(ldict[key]):
        if word not in index_table:
            index_table[word]=[]
            index_table[word].append(key)
        else:
            index_table[word].append(key)
            
# print(index_table.keys())



In [ ]:
for x in index_table.keys():
    unique_elements, counts_elements = np.unique(index_table[x], return_counts=True)
    if counts_elements.max() > 1:
        print(x)



In [ ]:
pindex_table={}
for key in ldict.keys():
    count=0
    for word in ldict[key]:
        count+=1
        if word not in pindex_table:
            pindex_table[word]={}
            pindex_table[word][key]=[]
            pindex_table[word][key].append(count)
        else:
            if key not in pindex_table[word]:
                pindex_table[word][key]=[]
            pindex_table[word][key].append(count)

In [ ]:
# print(pindex_table)

In [ ]:
words = pindex_table.keys()
print(words)

In [ ]:
# for key in words:
#     print("##### new word ######")
#     print("key  :  ",key)
#     print(pindex_table[key])
    

In [ ]:
query=input("Enter the query ")
query=query.replace(".","").replace("n't"," not").replace("]"," ").replace("["," ").replace(","," ").replace("?","").replace("/"," / ").split()
stemmedQuery=[porter.stem(x.lower()) for x in query]
QueryWithoutStoplist = [x for x in stemmedQuery if x not in swl]
print(QueryWithoutStoplist)

In [ ]:
#check for proximity query

ProximityValue=0
if '/' in stemmedQuery:
    ProximityValue = int(QueryWithoutStoplist[QueryWithoutStoplist.index('/')+1])
    w1=QueryWithoutStoplist[0]
    w2=QueryWithoutStoplist[1]
    
# elif 'not','and','or' in stemmedQuery:
#      print("its a boolean query")

In [ ]:
#proximity query function

def ProximityQueryProcessing(w1, w2, ProximityValue, pindex_table):
    word1={}
    word2={}
    for word in pindex_table.keys():
        if word == w1:
            for docid in pindex_table[word]:
                word1[docid]=pindex_table[word][docid]
        elif word==w2:
            for docid in pindex_table[word]:
                word2[docid]=pindex_table[word][docid]
                
    s1=set(word1.keys())
    s2=set(word2.keys())
    commonDoc=None
    commonDoc=s1.intersection(s2)
    print("s1 : ",s1,"\n\ns2 : ",s2)
    print("\n\n## Common in : ",commonDoc,"\n")
    listOfTrueDoc=[]
    if commonDoc is not None:
        for commonDoc in commonDoc:
                x=bool(checkPositions(word1[commonDoc],word2[commonDoc], ProximityValue,commonDoc))
                if x==True:
                    listOfTrueDoc.append(commonDoc)
    else:
        print("result not found")
        
    print(listOfTrueDoc)

In [ ]:
def checkPositions(word1, word2, ProximityValue,commonDoc): 
    for pos1 in word1:
        for pos2 in word2:
            if (abs(pos1-pos2) - 1) <= ProximityValue:
                print("DOC--> ",commonDoc,"-->pos1(",pos1,") - pos2(",pos2,")",abs(pos1-pos2)-1)
                return True
            else:
                print("false statement : \n","DOC--> ",commonDoc,"-->pos1(",pos1,") - pos2(",pos2,")",abs(pos1-pos2)-1)
                return False

In [ ]:
if ProximityValue is not 0:
    ProximityQueryProcessing(w1,w2,ProximityValue,pindex_table)

In [ ]:
# if AND, OR, NOT are present then it's boolean (complex boolean query search is compulsory)
# phrasal query i.e: 'running after'  (phrasal search is optional)

